In [1]:
import numpy as np
import pandas as pd

import geopy.distance

import shutil
import os
from collections import OrderedDict

import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots

import time

In [2]:
# Testing
from Final_Analysis_Scripts.CitySelection import CitySelection_Script
tier_1_2_cities = [
    'Ahmedabad', 'Bengaluru', 'Mumbai', 'Pune', 'Chennai', 'Hyderabad', 'Kolkata', 'Delhi', 'Visakhapatnam', 'Guwahati', 'Patna',
    'Raipur', 'Gurugram', 'Shimla', 'Jamshedpur', 'Thiruvananthapuram', 'Bhopal', 'Bhubaneswar', 'Amritsar', 'Jaipur', 'Lucknow', 'Dehradun'
]
from Final_Analysis_Scripts.PreProcessor import PreProcessor
preprocessor = PreProcessor(tier_1_2_cities, "./PreProcessed_Datasets")
general_params = {
    'PRESENT_YEAR': 2023,
    'FORECAST_YEAR': 2033,
    'SAMPLE_NAME': 'Sample1'
}
most_growth_cities, airports = CitySelection_Script(general_params, preprocessor, tier_1_2_cities, './Final_Analysis_Scripts/Temporary_Outputs', './Final_Analysis_Scripts/Temporary_Outputs')
print(airports)
print([airports[x].airport_info for x in airports])
print(most_growth_cities)
selected_city = [x for x in most_growth_cities][0]

selected_city = selected_city
AIRPORTS = airports
general_params = general_params
preprocessor = preprocessor
tier_1_2_cities = tier_1_2_cities
output_save_path = './Analysis_Outputs'
plotly_save_path = './Analysis_Outputs'

**************************************
Loading PreProcessed Datasets
Loaded AirRouteDatasets
Loaded Domestic Passenger Data
Loaded Indian Railways Data
Loaded Socio-Economic Data
Loaded Monument Visitors Data
**************************************


/home/sakshat/Documents/R2_Data_Labs_Hackathon/Final_Analysis_Scripts/CitySelection.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/sakshat/Documents/R2_Data_Labs_Hackathon/Final_Analysis_Scripts/CitySelection.py:114: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/sakshat/Documents/R2_Data_Labs_Hackathon/Final_Analysis_Scripts/CitySelection.py:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/sakshat/Documents/R

{'DEL': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dae48d60>, 'JAI': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7c40>, 'LKO': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea78b0>, 'CCU': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7f10>, 'BBI': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea75b0>, 'HYD': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7b20>, 'MAA': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7520>, 'BLR': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7be0>, 'PNQ': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea76d0>, 'BOM': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7250>, 'VTZ': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7670>, 'GAU': <Final_Analysis_Scripts.CitySelection.Airport object at 0x7fc6dcea7df0>}
[{'Name': 'DEL', 'City/Town': 'New Delh

In [3]:
PRESENT_YEAR = general_params['PRESENT_YEAR']
FORECAST_YEAR = general_params['FORECAST_YEAR']
SAMPLE_NAME = general_params['SAMPLE_NAME']

preprocessor.network_data = preprocessor.all_samples_network_data[SAMPLE_NAME]

In [4]:
distance_time_curve_fit = np.polyfit(preprocessor.all_network_data['Distance'].values, preprocessor.all_network_data['Time'].values, 1)

In [5]:
def get_route_passenger_traffic_data(cities):
    city_to_city_mapping = dict(zip(preprocessor.city_mapping['DomesticPassengerTraffic_City'], preprocessor.city_mapping['City']))
    route_traffic = []
    for idx, row in preprocessor.total_domestic_data.iterrows():
        if((row['FROM'] in city_to_city_mapping) and (row['TO'] in city_to_city_mapping)):
            route_traffic.append([city_to_city_mapping[row['FROM']], city_to_city_mapping[row['TO']], row['PASSENGERS']])
    route_traffic_df = pd.DataFrame(route_traffic, columns = ['From', 'To', 'Passengers_Target'])
    return route_traffic_df

route_traffic_df = get_route_passenger_traffic_data(tier_1_2_cities)
route_traffic_df['Year'] = pd.Series([PRESENT_YEAR] * route_traffic_df.shape[0])
route_traffic_df['Connecting'] = pd.Series([''] * route_traffic_df.shape[0])

In [6]:
def get_railways_info_features(raw_route_traffic_df):
    route_traffic_df = raw_route_traffic_df.copy()
    railway_info = []
    city_to_district_mapping = dict(zip(preprocessor.city_mapping['City'].values, preprocessor.city_mapping['StationCodeData_District'].values))
    for idx, row in route_traffic_df.iterrows():
        from_district = city_to_district_mapping[row['From']]
        to_district = city_to_district_mapping[row['To']]
        if(from_district not in preprocessor.city_to_city_train_dict):
            railway_info.append([row['From'], row['To'], row['Year']] + [0, np.nan, np.nan])
        else:
            if(to_district not in preprocessor.city_to_city_train_dict[from_district]):
                railway_info.append([row['From'], row['To'], row['Year']] + [0, np.nan, np.nan])
            else:
                route_railway_info = preprocessor.city_to_city_train_dict[from_district][to_district]
                num_trains = len(route_railway_info)
                avg_duration = np.nanmean([x['duration'] for x in route_railway_info])
                third_ac = np.nansum([x['third_ac'] for x in route_railway_info])
                chair_car = np.nansum([x['chair_car'] for x in route_railway_info])
                first_class = np.nansum([x['first_class'] for x in route_railway_info])
                sleeper = np.nansum([x['sleeper'] for x in route_railway_info])
                second_ac = np.nansum([x['second_ac'] for x in route_railway_info])
                first_ac = np.nansum([x['first_ac'] for x in route_railway_info])
                # Based on https://en.wikipedia.org/wiki/Indian_Railways_coaching_stock & https://www.quora.com/What-is-the-capacity-of-normal-Indian-passenger-train
                #   first_ac -> 20 (1x20)
                #   second_ac -> 100 (2x50)
                #   third_ax -> 180 (3x60)
                #   chair_car -> 150 (75x2)
                #   sleeper -> 840 (12x70)
                #   first_class -> 20 (1x20)
                capacity = (
                    first_ac * 20 +
                    second_ac * 100 +
                    third_ac * 180 +
                    chair_car * 150 +
                    sleeper * 840 +
                    first_class * 20
                )
                # Forecasting growth of railways
                #    -> Based on https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiQp_S3qNP9AhXIwTgGHYuaB5EQFnoECA8QAQ&url=https%3A%2F%2Findianrailways.gov.in%2Frailwayboard%2Fuploads%2Fdirectorate%2Fstat_econ%2FAnnual-Reports-2020-2021%2FAnnual-Report-English.pdf&usg=AOvVaw2YMsSpEfqTOjBD13N-ZsJl,
                #    -> On average, over the past decade, number of passenger coaches increased by ~1000 per year while total running track increased by ~1000 km per year
                #    -> On average, growth rate in railway connectivity & passenger capacity was ~2% per year
                num_trains = num_trains + num_trains * (2 / 100) * (row['Year'] - PRESENT_YEAR)
                capacity = capacity + capacity * (2 / 100) * (row['Year'] - PRESENT_YEAR)
                railway_info.append([row['From'], row['To'], row['Year']] + [num_trains, avg_duration, capacity])
    railway_info_df = pd.DataFrame(railway_info, columns = ['From', 'To', 'Year', 'NumTrains_Railways', 'Duration_Railways', 'Capacity_Railways'])
    route_traffic_df = pd.merge(route_traffic_df, railway_info_df, on = ['From', 'To', 'Year'], how = 'left')
    return route_traffic_df

route_traffic_df = get_railways_info_features(route_traffic_df)

In [7]:
OVERLAY_TIME = 120
airport_to_coords_mapping = dict(zip(preprocessor.city_mapping['AirRouteData_AirportCode'].values, preprocessor.city_mapping['Airport_City_Coords'].values))
def get_timing(from_airport, to_airport):
        route_timing_data = preprocessor.all_network_data[(preprocessor.all_network_data['From'] == from_airport) & (preprocessor.all_network_data['To'] == to_airport)]
        if(route_timing_data.shape[0] == 0):
            from_airport_coords = airport_to_coords_mapping[from_airport]
            from_airport_coords_lat = float(from_airport_coords.split(',')[0])
            from_airport_coords_lon = float(from_airport_coords.split(',')[1])
            from_airport_coords = (from_airport_coords_lat, from_airport_coords_lon)
            to_airport_coords = airport_to_coords_mapping[to_airport]
            to_airport_coords_lat = float(to_airport_coords.split(',')[0])
            to_airport_coords_lon = float(to_airport_coords.split(',')[1])
            to_airport_coords = (to_airport_coords_lat, to_airport_coords_lon)
            route_distance = geopy.distance.geodesic(from_airport_coords, to_airport_coords).miles
            route_timing = route_distance * distance_time_curve_fit[0] + distance_time_curve_fit[1]
        else:
            assert(route_timing_data['Time'].nunique() == 1)
            route_distance = route_timing_data['Distance'].mean()
            route_timing = route_timing_data['Time'].mean()
        return route_timing, route_distance

def get_route_timing_features(raw_route_traffic_df):

    route_traffic_df = raw_route_traffic_df.copy()
    route_timings = []
    city_to_airport_mapping = dict(zip(preprocessor.city_mapping['City'].values, preprocessor.city_mapping['AirRouteData_AirportCode'].values))
    for idx, row in route_traffic_df.iterrows():
        from_airport = city_to_airport_mapping[row['From']]
        to_airport = city_to_airport_mapping[row['To']]
        connecting = row['Connecting']
        if(connecting == ''):
            route_timing = get_timing(from_airport, to_airport)[0]
        else:
            connecting_airport = city_to_airport_mapping[connecting]
            route_timing = get_timing(from_airport, connecting_airport)[0] + get_timing(connecting_airport, to_airport)[0] + OVERLAY_TIME
        route_timings.append(route_timing)
    route_traffic_df['Duration_AirRoute'] = pd.Series(route_timings)
    route_traffic_df['NumTrains_Railways'] = route_traffic_df['NumTrains_Railways'].fillna(0)
    return route_traffic_df

route_traffic_df = get_route_timing_features(route_traffic_df)

In [8]:
pca_vals_df = preprocessor.present_features
pca_cols = ['City_' + x for x in pca_vals_df.columns if x != 'City']
pca_vals_df.columns = pca_cols + ['City']
pca_vals_df['Year'] = pd.Series([PRESENT_YEAR] * pca_vals_df.shape[0])

pca_vals_forecasted_df = {}
for year in range(PRESENT_YEAR + 1, FORECAST_YEAR + 1):
    pca_vals_forecasted_year_df = pd.read_csv(f"{output_save_path}/Forecasted_Features/{year}.csv")
    pca_vals_forecasted_year_df.columns = ['City_' + x if x != 'City' else x for x in pca_vals_forecasted_year_df.columns]
    pca_vals_forecasted_year_df['Year'] = pd.Series([year] * pca_vals_forecasted_year_df.shape[0])
    pca_vals_forecasted_df[year] = pca_vals_forecasted_year_df

In [9]:
pca_cols

['City_economic_pca1',
 'City_economic_pca2',
 'City_tourism_pca1',
 'City_tourism_pca2',
 'City_education_pca1',
 'City_education_pca2',
 'City_population_pca1',
 'City_population_pca2']

In [10]:
total_pca_vals_df = pca_vals_df
for year in pca_vals_forecasted_df:
    total_pca_vals_df = pd.concat([total_pca_vals_df, pca_vals_forecasted_df[year]], axis = 0)

def get_pca_vals_features(raw_route_traffic_df):
    route_traffic_df = raw_route_traffic_df.copy()
    route_traffic_df = pd.merge(route_traffic_df, total_pca_vals_df, left_on = ['From', 'Year'], right_on = ['City', 'Year'], how = 'left')
    route_traffic_df = pd.merge(route_traffic_df, total_pca_vals_df, left_on = ['To', 'Year'], right_on = ['City', 'Year'], how = 'left', suffixes = ('_pca_vals_FROM', '_pca_vals_TO'))

    route_traffic_df = route_traffic_df.drop(['City_pca_vals_TO', 'City_pca_vals_FROM'], axis = 1)
    for col in pca_cols:
        route_traffic_df[f'{col}_avg_PCA'] = (route_traffic_df[f'{col}_pca_vals_FROM'] + route_traffic_df[f'{col}_pca_vals_TO']) / 2
        route_traffic_df[f'{col}_diff_PCA'] = (route_traffic_df[f'{col}_pca_vals_FROM'] - route_traffic_df[f'{col}_pca_vals_TO'])
        route_traffic_df = route_traffic_df.drop([f'{col}_pca_vals_FROM', f'{col}_pca_vals_TO', f'{col}_pca_vals_FROM', f'{col}_pca_vals_TO'], axis = 1)

    return route_traffic_df

route_traffic_df = get_pca_vals_features(route_traffic_df)

In [11]:
route_traffic_df.head()

,From,To,Passengers_Target,Year,Connecting,NumTrains_Railways,Duration_Railways,Capacity_Railways,Duration_AirRoute,City_economic_pca1_avg_PCA,...,City_tourism_pca2_avg_PCA,City_tourism_pca2_diff_PCA,City_education_pca1_avg_PCA,City_education_pca1_diff_PCA,City_education_pca2_avg_PCA,City_education_pca2_diff_PCA,City_population_pca1_avg_PCA,City_population_pca1_diff_PCA,City_population_pca2_avg_PCA,City_population_pca2_diff_PCA
0,Ahmedabad,Amritsar,13033.0,2023,,0.0,NaN,NaN,115.147624,-0.275142,...,-0.058844,-0.268289,-1.674012,5.529328,-0.278629,-0.190334,-0.104003,3.104983,-0.060905,0.676665
1,Ahmedabad,Bengaluru,405751.0,2023,,3.0,2044.650000,3360.0,130.000000,0.628319,...,-0.166174,-0.053629,4.358537,-6.535770,-0.639212,0.530832,2.707758,-2.518539,-0.070452,0.695759
2,Ahmedabad,Bhopal,20514.0,2023,,2.0,914.930556,2240.0,85.000000,-0.325844,...,-0.641350,0.896723,-1.496097,5.173497,-0.043189,-0.661215,-0.201694,3.300365,0.286072,-0.017291
3,Ahmedabad,Bhubaneswar,16866.0,2023,,0.0,NaN,NaN,125.000000,-0.292547,...,-0.219295,0.052613,-1.575576,5.332457,-0.151629,-0.444334,-0.045728,2.988433,-0.603582,1.762018
4,Ahmedabad,Chennai,139900.0,2023,,0.0,NaN,NaN,140.000000,-0.213579,...,1.793267,-3.972511,0.562076,1.057151,0.067749,-0.883091,0.566487,1.764003,1.447931,-2.341008


In [12]:
y_features = [x for x in route_traffic_df.columns if x.endswith('_Target')]
X_features = [x for x in route_traffic_df.columns if (x.endswith('_PCA')) or (x.endswith('_AirRoute')) or (x.endswith('_Railways'))]

In [13]:
route_traffic_df.shape

(286, 25)

In [14]:
# Trimming data for model training - Removing routes having no target variable
# Standardizing data
valid_route_traffic_df = route_traffic_df.copy()
cols_standardization_vals = {}
for col_idx, col in enumerate(valid_route_traffic_df.columns):
    if((col in X_features) or (col in y_features)):
        if(col in y_features):
            to_drop_idx = pd.isnull(valid_route_traffic_df[col])
            to_drop_idx = to_drop_idx[to_drop_idx == True].index
            valid_route_traffic_df = valid_route_traffic_df.drop(to_drop_idx, axis = 0)
        elif(col in X_features):
            col_mean = np.nanmean(valid_route_traffic_df[col].values)
            valid_route_traffic_df[col] = valid_route_traffic_df[col].fillna(col_mean)
        col_mean = valid_route_traffic_df[col].mean()
        col_std = valid_route_traffic_df[col].std()
        valid_route_traffic_df[col] = (valid_route_traffic_df[col] - col_mean) / (col_std + 1e-20)
        cols_standardization_vals[col] = {'mean': col_mean, 'std': col_std}

data_X = valid_route_traffic_df[X_features].values
data_y = valid_route_traffic_df[y_features].values[:, 0]
data_cities = [(x[0], x[1]) for x in valid_route_traffic_df[['From', 'To']].values]

In [15]:
class LinearModel:
    def __init__(self, coefs):
        self.coefs = coefs
    def predict(self, X):
        assert(len(self.coefs) == X.shape[1] + 1)
        return np.dot([*self.coefs.values()][1:], X.transpose()) + self.coefs['intercept']

model = LinearModel(preprocessor.RouteSelection_model_coefs)

uniq_hubs = preprocessor.network_data[preprocessor.network_data['FromHub'] == 1]['From'].unique()

# get_route_demand_forecasts - split

In [16]:
# SELECTED_CITY = 'Bhopal'
# SELECTED_HUB_AIRPORT = 'CCU'

In [17]:
# selected_hub_city = preprocessor.city_mapping[preprocessor.city_mapping['AirRouteData_AirportCode'] == SELECTED_HUB_AIRPORT].iloc[0]['City']
# connecting_airports = []
# for airport in AIRPORTS[SELECTED_HUB_AIRPORT].to_airport_list:
#     if(airport not in connecting_airports):
#         connecting_airports.append(airport)
# for airport in AIRPORTS[SELECTED_HUB_AIRPORT].from_airport_list:
#     if(airport not in connecting_airports):
#         connecting_airports.append(airport)

In [18]:
# expected_route_traffic_df = []
# for year in np.arange(PRESENT_YEAR, FORECAST_YEAR + 1):
#     expected_route_traffic_df.append([SELECTED_CITY, selected_hub_city, year, ''])
#     airport_to_city_mapping = dict(zip(preprocessor.city_mapping['AirRouteData_AirportCode'].values, preprocessor.city_mapping['City'].values))
#     for airport in connecting_airports:
#         expected_route_traffic_df.append([SELECTED_CITY, airport_to_city_mapping[airport.airport_info['Name']], year, selected_hub_city])
#     expected_route_traffic_df.append([selected_hub_city, SELECTED_CITY, year, ''])
#     airport_to_city_mapping = dict(zip(preprocessor.city_mapping['AirRouteData_AirportCode'].values, preprocessor.city_mapping['City'].values))
#     for airport in connecting_airports:
#         expected_route_traffic_df.append([airport_to_city_mapping[airport.airport_info['Name']], SELECTED_CITY, year, selected_hub_city])
# expected_route_traffic_df = pd.DataFrame(expected_route_traffic_df, columns = ['From', 'To', 'Year', 'Connecting'])

In [19]:
# expected_route_traffic_df = get_railways_info_features(expected_route_traffic_df)
# expected_route_traffic_df = get_route_timing_features(expected_route_traffic_df)
# expected_route_traffic_df = get_pca_vals_features(expected_route_traffic_df)
# duration_in = expected_route_traffic_df[(expected_route_traffic_df['From'] == selected_hub_city) & (expected_route_traffic_df['To'] == SELECTED_CITY)].iloc[0]['Duration_AirRoute']
# duration_out = expected_route_traffic_df[(expected_route_traffic_df['To'] == selected_hub_city) & (expected_route_traffic_df['From'] == SELECTED_CITY)].iloc[0]['Duration_AirRoute']

In [20]:
# expected_route_traffic_df.head()

In [21]:
# railway_info_out = expected_route_traffic_df[(expected_route_traffic_df['From'] == SELECTED_CITY) & (expected_route_traffic_df['To'] == selected_hub_city) & (expected_route_traffic_df['Year'] == PRESENT_YEAR)].iloc[0]
# railway_info_in = expected_route_traffic_df[(expected_route_traffic_df['From'] == selected_hub_city) & (expected_route_traffic_df['To'] == SELECTED_CITY) & (expected_route_traffic_df['Year'] == PRESENT_YEAR)].iloc[0]
# railway_num_out = railway_info_out['NumTrains_Railways']
# railway_num_in = railway_info_in['NumTrains_Railways']
# railway_duration_out = railway_info_out['Duration_Railways']
# railway_duration_in = railway_info_in['Duration_Railways']
# railway_capacity_out = railway_info_out['Capacity_Railways']
# railway_capacity_in = railway_info_in['Capacity_Railways']

# def check_if_both_nan(x, y):
#     if((pd.isnull(x)) & (pd.isnull(y))):
#         return "N/A"
#     else:
#         if(pd.isnull(x)):
#             return y
#         elif(pd.isnull(y)):
#             return x
#         else:
#             return (x + y) // 2
# railway_num = check_if_both_nan(railway_num_in, railway_num_out)
# railway_duration = check_if_both_nan(railway_duration_in, railway_duration_out)
# railway_capacity = check_if_both_nan(railway_capacity_in, railway_capacity_out)

# for col_idx, col in enumerate(expected_route_traffic_df.columns):
#     if(col in X_features):
#         col_mean = cols_standardization_vals[col]['mean']
#         col_std = cols_standardization_vals[col]['std']
#         expected_route_traffic_df[col] = expected_route_traffic_df[col].fillna(col_mean)
#         expected_route_traffic_df[col] = (expected_route_traffic_df[col] - col_mean) / (col_std + 1e-20)

In [22]:
# data_X = expected_route_traffic_df[X_features]
# target_mean = cols_standardization_vals[y_features[0]]['mean']
# target_std = cols_standardization_vals[y_features[0]]['std']
# pred = model.predict(data_X) * target_std + (target_mean)
# expected_route_traffic_df['ForecastedDemand'] = pd.Series(pred)

In [23]:
# # Domestic Passenger Traffic would include local + connecting demand, hence excluding adding connecting demand
# CONNECTING_DEMAND_FACTOR = 0
# def adjust_connecting_demand(row):
#     if(row['Connecting'] == ''):
#         return row['ForecastedDemand']
#     else:
#         return row['ForecastedDemand'] * CONNECTING_DEMAND_FACTOR
# expected_route_traffic_df['AdjustedForecastedDemand'] = expected_route_traffic_df.apply(adjust_connecting_demand, axis = 1)

# # Domestic Passenger Traffic would include local + connecting demand, hence excluding adding connecting demand
# CONNECTING_DEMAND_FACTOR = 0
# def adjust_connecting_demand(row):
#     if(row['Connecting'] == ''):
#         return row['ForecastedDemand']
#     else:
#         return row['ForecastedDemand'] * CONNECTING_DEMAND_FACTOR
# expected_route_traffic_df['AdjustedForecastedDemand'] = expected_route_traffic_df.apply(adjust_connecting_demand, axis = 1)

In [24]:
# in_total_traffic = expected_route_traffic_df[expected_route_traffic_df['To'] == SELECTED_CITY]
# in_total_traffic = in_total_traffic.groupby('Year')['AdjustedForecastedDemand'].aggregate('sum').reset_index(drop = False)
# out_total_traffic = expected_route_traffic_df[expected_route_traffic_df['From'] == SELECTED_CITY]
# out_total_traffic = out_total_traffic.groupby('Year')['AdjustedForecastedDemand'].aggregate('sum').reset_index(drop = False)
# in_out_total_traffic = pd.merge(in_total_traffic, out_total_traffic, on = 'Year', suffixes = ('_InTraffic', '_OutTraffic'))

In [25]:
# actual_present_out_traffic = valid_route_traffic_df[(valid_route_traffic_df['From'] == SELECTED_CITY) & (valid_route_traffic_df['To'] == selected_hub_city)]
# if(actual_present_out_traffic.shape[0] > 0):
#     assert(actual_present_out_traffic.shape[0] == 1)
#     actual_present_out_traffic = actual_present_out_traffic.iloc[0]['Passengers_Target'] * target_std + target_mean
#     out_traffic_adjustment = actual_present_out_traffic - in_out_total_traffic.iloc[0]['AdjustedForecastedDemand_OutTraffic']
#     in_out_total_traffic['AdjustedForecastedDemand_OutTraffic'] = in_out_total_traffic['AdjustedForecastedDemand_OutTraffic'] + out_traffic_adjustment
# actual_present_in_traffic = valid_route_traffic_df[(valid_route_traffic_df['To'] == SELECTED_CITY) & (valid_route_traffic_df['From'] == selected_hub_city)]
# if(actual_present_in_traffic.shape[0] > 0):
#     actual_present_in_traffic = actual_present_in_traffic.iloc[0]['Passengers_Target'] * target_std + target_mean
#     in_traffic_adjustment = actual_present_in_traffic - in_out_total_traffic.iloc[0]['AdjustedForecastedDemand_InTraffic']
#     in_out_total_traffic['AdjustedForecastedDemand_InTraffic'] = in_out_total_traffic['AdjustedForecastedDemand_InTraffic'] + in_traffic_adjustment

In [26]:
# in_out_total_traffic.to_csv(f"{output_save_path}/Forecasted_Route_Demand/City{SELECTED_CITY}_Hub{SELECTED_HUB_AIRPORT}.csv", index = None)

In [27]:
# in_out_total_traffic, (duration_in, duration_out)

# get_route_demand_forecasts - full

In [28]:
def get_route_demand_forecasts(SELECTED_CITY, SELECTED_HUB_AIRPORT):
    selected_hub_city = preprocessor.city_mapping[preprocessor.city_mapping['AirRouteData_AirportCode'] == SELECTED_HUB_AIRPORT].iloc[0]['City']
    connecting_airports = []
    for airport in AIRPORTS[SELECTED_HUB_AIRPORT].to_airport_list:
        if(airport not in connecting_airports):
            connecting_airports.append(airport)
    for airport in AIRPORTS[SELECTED_HUB_AIRPORT].from_airport_list:
        if(airport not in connecting_airports):
            connecting_airports.append(airport)

    expected_route_traffic_df = []
    for year in np.arange(PRESENT_YEAR, FORECAST_YEAR + 1):
        expected_route_traffic_df.append([SELECTED_CITY, selected_hub_city, year, ''])
        airport_to_city_mapping = dict(zip(preprocessor.city_mapping['AirRouteData_AirportCode'].values, preprocessor.city_mapping['City'].values))
        for airport in connecting_airports:
            expected_route_traffic_df.append([SELECTED_CITY, airport_to_city_mapping[airport.airport_info['Name']], year, selected_hub_city])
        expected_route_traffic_df.append([selected_hub_city, SELECTED_CITY, year, ''])
        airport_to_city_mapping = dict(zip(preprocessor.city_mapping['AirRouteData_AirportCode'].values, preprocessor.city_mapping['City'].values))
        for airport in connecting_airports:
            expected_route_traffic_df.append([airport_to_city_mapping[airport.airport_info['Name']], SELECTED_CITY, year, selected_hub_city])
    expected_route_traffic_df = pd.DataFrame(expected_route_traffic_df, columns = ['From', 'To', 'Year', 'Connecting'])

    expected_route_traffic_df = get_railways_info_features(expected_route_traffic_df)
    expected_route_traffic_df = get_route_timing_features(expected_route_traffic_df)
    expected_route_traffic_df = get_pca_vals_features(expected_route_traffic_df)
    duration_in = expected_route_traffic_df[(expected_route_traffic_df['From'] == selected_hub_city) & (expected_route_traffic_df['To'] == SELECTED_CITY)].iloc[0]['Duration_AirRoute']
    duration_out = expected_route_traffic_df[(expected_route_traffic_df['To'] == selected_hub_city) & (expected_route_traffic_df['From'] == SELECTED_CITY)].iloc[0]['Duration_AirRoute']

    railway_info_out = expected_route_traffic_df[(expected_route_traffic_df['From'] == SELECTED_CITY) & (expected_route_traffic_df['To'] == selected_hub_city) & (expected_route_traffic_df['Year'] == PRESENT_YEAR)].iloc[0]
    railway_info_in = expected_route_traffic_df[(expected_route_traffic_df['From'] == selected_hub_city) & (expected_route_traffic_df['To'] == SELECTED_CITY) & (expected_route_traffic_df['Year'] == PRESENT_YEAR)].iloc[0]
    railway_num_out = railway_info_out['NumTrains_Railways']
    railway_num_in = railway_info_in['NumTrains_Railways']
    railway_duration_out = railway_info_out['Duration_Railways']
    railway_duration_in = railway_info_in['Duration_Railways']
    railway_capacity_out = railway_info_out['Capacity_Railways']
    railway_capacity_in = railway_info_in['Capacity_Railways']

    def check_if_both_nan(x, y):
        if((pd.isnull(x)) & (pd.isnull(y))):
            return "N/A"
        else:
            if(pd.isnull(x)):
                return y
            elif(pd.isnull(y)):
                return x
            else:
                return (x + y) // 2
    railway_num = check_if_both_nan(railway_num_in, railway_num_out)
    railway_duration = check_if_both_nan(railway_duration_in, railway_duration_out)
    railway_capacity = check_if_both_nan(railway_capacity_in, railway_capacity_out)
#     if((railway_capacity != 'N/A')):
#         if(railway_duration != 'N/A'):
#             railway_capacity = railway_capacity / railway_duration * (24 * 60)
#         else:
#             railway_capacity = 'N/A'
    
    for col_idx, col in enumerate(expected_route_traffic_df.columns):
        if(col in X_features):
            col_mean = cols_standardization_vals[col]['mean']
            col_std = cols_standardization_vals[col]['std']
            expected_route_traffic_df[col] = expected_route_traffic_df[col].fillna(col_mean)
            expected_route_traffic_df[col] = (expected_route_traffic_df[col] - col_mean) / (col_std + 1e-20)

    data_X = expected_route_traffic_df[X_features]
    target_mean = cols_standardization_vals[y_features[0]]['mean']
    target_std = cols_standardization_vals[y_features[0]]['std']
    pred = model.predict(data_X) * target_std + (target_mean)
    expected_route_traffic_df['ForecastedDemand'] = pd.Series(pred)

    # Domestic Passenger Traffic would include local + connecting demand, hence excluding adding connecting demand
    CONNECTING_DEMAND_FACTOR = 0
    def adjust_connecting_demand(row):
        if(row['Connecting'] == ''):
            return row['ForecastedDemand']
        else:
            return row['ForecastedDemand'] * CONNECTING_DEMAND_FACTOR
    expected_route_traffic_df['AdjustedForecastedDemand'] = expected_route_traffic_df.apply(adjust_connecting_demand, axis = 1)

    # Domestic Passenger Traffic would include local + connecting demand, hence excluding adding connecting demand
    CONNECTING_DEMAND_FACTOR = 0
    def adjust_connecting_demand(row):
        if(row['Connecting'] == ''):
            return row['ForecastedDemand']
        else:
            return row['ForecastedDemand'] * CONNECTING_DEMAND_FACTOR
    expected_route_traffic_df['AdjustedForecastedDemand'] = expected_route_traffic_df.apply(adjust_connecting_demand, axis = 1)

    in_total_traffic = expected_route_traffic_df[expected_route_traffic_df['To'] == SELECTED_CITY]
    in_total_traffic = in_total_traffic.groupby('Year')['AdjustedForecastedDemand'].aggregate('sum').reset_index(drop = False)
    out_total_traffic = expected_route_traffic_df[expected_route_traffic_df['From'] == SELECTED_CITY]
    out_total_traffic = out_total_traffic.groupby('Year')['AdjustedForecastedDemand'].aggregate('sum').reset_index(drop = False)
    in_out_total_traffic = pd.merge(in_total_traffic, out_total_traffic, on = 'Year', suffixes = ('_InTraffic', '_OutTraffic'))

    actual_present_out_traffic = valid_route_traffic_df[(valid_route_traffic_df['From'] == SELECTED_CITY) & (valid_route_traffic_df['To'] == selected_hub_city)]
    if(actual_present_out_traffic.shape[0] > 0):
        assert(actual_present_out_traffic.shape[0] == 1)
        actual_present_out_traffic = actual_present_out_traffic.iloc[0]['Passengers_Target'] * target_std + target_mean
        out_traffic_adjustment = actual_present_out_traffic - in_out_total_traffic.iloc[0]['AdjustedForecastedDemand_OutTraffic']
        in_out_total_traffic['AdjustedForecastedDemand_OutTraffic'] = in_out_total_traffic['AdjustedForecastedDemand_OutTraffic'] + out_traffic_adjustment
    actual_present_in_traffic = valid_route_traffic_df[(valid_route_traffic_df['To'] == SELECTED_CITY) & (valid_route_traffic_df['From'] == selected_hub_city)]
    if(actual_present_in_traffic.shape[0] > 0):
        actual_present_in_traffic = actual_present_in_traffic.iloc[0]['Passengers_Target'] * target_std + target_mean
        in_traffic_adjustment = actual_present_in_traffic - in_out_total_traffic.iloc[0]['AdjustedForecastedDemand_InTraffic']
        in_out_total_traffic['AdjustedForecastedDemand_InTraffic'] = in_out_total_traffic['AdjustedForecastedDemand_InTraffic'] + in_traffic_adjustment

    in_out_total_traffic.to_csv(f"{output_save_path}/Forecasted_Route_Demand/City{SELECTED_CITY}_Hub{SELECTED_HUB_AIRPORT}.csv", index = None)
    return in_out_total_traffic, (duration_in, duration_out, railway_num, railway_duration, railway_capacity)

In [29]:
airport_to_city_mapping = dict(zip(preprocessor.city_mapping['AirRouteData_AirportCode'].values, preprocessor.city_mapping['City'].values))
city_to_airport_mapping = dict(zip(preprocessor.city_mapping['City'].values, preprocessor.city_mapping['AirRouteData_AirportCode'].values))
all_airports_in_network = [airport_to_city_mapping[AIRPORTS[x].airport_info['Name']] for x in AIRPORTS]

In [30]:
shutil.rmtree(f"{output_save_path}/Forecasted_Route_Demand/", ignore_errors = True)
os.mkdir(f"{output_save_path}/Forecasted_Route_Demand/")
route_info_df = []
city = selected_city

In [31]:
route_forecasted_demands_dict = {}

for hub in uniq_hubs:
    SELECTED_CITY = city
    SELECTED_HUB_AIRPORT = hub
    route_forecasted_demands, durations_railway = get_route_demand_forecasts(SELECTED_CITY, SELECTED_HUB_AIRPORT)
    assert(route_forecasted_demands.iloc[0]['Year'] == PRESENT_YEAR)
    assert(route_forecasted_demands.iloc[route_forecasted_demands.shape[0] - 1]['Year'] == FORECAST_YEAR)
    route_forecasted_demands_dict[f'{SELECTED_CITY}-{SELECTED_HUB_AIRPORT}'] = route_forecasted_demands

    selected_city_airport = city_to_airport_mapping[SELECTED_CITY]
    route_network_data_out = preprocessor.all_network_data[(preprocessor.all_network_data['From'] == selected_city_airport) & (preprocessor.all_network_data['To'] == SELECTED_HUB_AIRPORT)]
    route_network_data_in = preprocessor.all_network_data[(preprocessor.all_network_data['To'] == selected_city_airport) & (preprocessor.all_network_data['From'] == SELECTED_HUB_AIRPORT)]
    if(route_network_data_out.shape[0] == 0):
        NUM_OUT_MARKET = 0
        PRICE_OUT_MARKET = "N/A"
    else:
        PRICE_OUT_MARKET = f"${int(route_network_data_out['Cheapest Price'].mean())}"
        NUM_OUT_MARKET = int(route_network_data_out['Number of Flights'].sum())
    if(route_network_data_in.shape[0] == 0):
        NUM_IN_MARKET = 0
        PRICE_IN_MARKET = "N/A"
    else:
        PRICE_IN_MARKET = f"${int(route_network_data_in['Cheapest Price'].mean())}"
        NUM_IN_MARKET = int(route_network_data_in['Number of Flights'].sum())

    DISTANCE = (get_timing(selected_city_airport, SELECTED_HUB_AIRPORT)[1] + get_timing(SELECTED_HUB_AIRPORT, selected_city_airport)[1]) / 2.0

    route_info_df.append([
        SELECTED_CITY, SELECTED_HUB_AIRPORT,
        durations_railway[0], durations_railway[1],
        durations_railway[2], durations_railway[3], durations_railway[4],
        route_forecasted_demands.iloc[0]['AdjustedForecastedDemand_InTraffic'],
        route_forecasted_demands.iloc[0]['AdjustedForecastedDemand_OutTraffic'],
        route_forecasted_demands.iloc[route_forecasted_demands.shape[0] - 1]['AdjustedForecastedDemand_InTraffic'],
        route_forecasted_demands.iloc[route_forecasted_demands.shape[0] - 1]['AdjustedForecastedDemand_OutTraffic'],
        NUM_OUT_MARKET, NUM_IN_MARKET,
        PRICE_OUT_MARKET, PRICE_IN_MARKET,
        DISTANCE
    ])

In [32]:
route_info_df = pd.DataFrame(route_info_df, columns = [
    'City', 'Hub',
    'IncomingFlightDuration', 'OutgoingFlightDuration',
    'RailwayNum', 'RailwayDuration', 'RailwayCapacity',
    'PresentYearInForecast', 'PresentYearOutForecast',
    'ForecastYearInForecast', 'ForecastYearOutForecast',
    'NUMBER_PLANES_OUT_MARKET', 'NUMBER_PLANES_IN_MARKET',
    'PRICE_OUT_MARKET', 'PRICE_IN_MARKET',
    'DISTANCE'
])

In [33]:
def plotly_RouteSelection(routes, demand_forecasts, plotly_save_path):
    
    for route in routes:

        city, hub = route.split('-')
        route_in_demand = demand_forecasts[route]['AdjustedForecastedDemand_InTraffic'].values[1:]
        route_out_demand = demand_forecasts[route]['AdjustedForecastedDemand_OutTraffic'].values[1:]
        years = demand_forecasts[route]['Year'].values[1:]
        
        fig1 = make_subplots(
            rows = 2, cols = 1,
            subplot_titles = [f"{city}→{hub}", f"{hub}→{city}"],
            shared_xaxes = True
        )
        
        fig1.add_trace(
            go.Bar(
                x = years, y = route_out_demand,
                hovertext = [f"Year: {x}<br>Passenger Forecast: {int(y)}" for x, y in zip(years, route_out_demand)],
                hoverinfo = 'text', marker = dict(color = '#2C88D9')
            ),
            row = 1, col = 1
        )
        
        fig1.add_trace(
            go.Bar(
                x = years, y = route_in_demand,
                hovertext = [f"Year: {x}<br>Passenger Forecast: {int(y)}" for x, y in zip(years, route_in_demand)],
                hoverinfo = 'text', marker = dict(color = '#2C88D9')
            ),
            row = 2, col = 1
        )
        
        fig1.update_layout(
            title_text = f"Forecasted Total Air-traffic Demand",
            height = 700, width = 500,
            paper_bgcolor = '#DBD8FD' , plot_bgcolor = '#DBD8FD',
            titlefont = dict(size = 20),
            showlegend = False
        )
        
        if(route == routes[0]):
            pyo.plot(fig1, output_type = 'file', filename = f'{plotly_save_path}/{route}_RouteSelection.html', config = {"displayModeBar": False, "showTips": False})
        
        div1 = pyo.plot(fig1, output_type = 'div', include_plotlyjs = False, show_link = False, link_text = "", config = {"displayModeBar": False, "showTips": False})
        with open(f'{plotly_save_path}/{route}_RouteSelection_Graph1.txt', 'w') as save_file:
            save_file.write(div1)

In [34]:
route_info_df['GrowthIn'] = (route_info_df['ForecastYearInForecast'] - route_info_df['PresentYearInForecast']) / (route_info_df['PresentYearInForecast'] + 1e-12) * 100
route_info_df['GrowthOut'] = (route_info_df['ForecastYearOutForecast'] - route_info_df['PresentYearOutForecast']) / (route_info_df['PresentYearOutForecast'] + 1e-12) * 100
route_info_df['AvgGrowth'] = (route_info_df['GrowthIn'] + route_info_df['GrowthOut']) / 2.0
route_info_df = route_info_df.sort_values('AvgGrowth', ascending = False)
route_info_df['Route'] = route_info_df.apply(lambda x: x['City'] + '-' + x['Hub'], axis = 1)
route_info_df = route_info_df.set_index('Route').head(5)
print([*route_info_df.index])

route_forecasted_demands_dict = dict([(x, route_forecasted_demands_dict[x]) for x in route_forecasted_demands_dict if x in [*route_info_df.index]])

plotly_RouteSelection([*route_info_df.index], route_forecasted_demands_dict, plotly_save_path)

route_info_df = OrderedDict(route_info_df.to_dict(orient = 'index'))

['Bhopal-BLR', 'Bhopal-HYD', 'Bhopal-DEL', 'Bhopal-CCU', 'Bhopal-BOM']


In [35]:
route_info_df

OrderedDict([('Bhopal-BLR',
              {'City': 'Bhopal',
               'Hub': 'BLR',
               'IncomingFlightDuration': 115.0,
               'OutgoingFlightDuration': 125.0,
               'RailwayNum': 12.0,
               'RailwayDuration': 1682.0,
               'RailwayCapacity': 9400.0,
               'PresentYearInForecast': 45554.0,
               'PresentYearOutForecast': 49042.0,
               'ForecastYearInForecast': 976569.406081995,
               'ForecastYearOutForecast': 1116445.2501216326,
               'NUMBER_PLANES_OUT_MARKET': 2,
               'NUMBER_PLANES_IN_MARKET': 2,
               'PRICE_OUT_MARKET': '$78',
               'PRICE_IN_MARKET': '$64',
               'DISTANCE': 697.0,
               'GrowthIn': 2043.7621418140996,
               'GrowthOut': 2176.5084012104576,
               'AvgGrowth': 2110.1352715122785}),
             ('Bhopal-HYD',
              {'City': 'Bhopal',
               'Hub': 'HYD',
               'IncomingFlightDu